In [1]:
import os, pandas as pd
import numpy as np


# Leer los archivos de datos en dataframes

country_isos=pd.read_excel(os.path.join('country_isos.xlsx'))
poblacion=pd.read_csv(os.path.join("co2.csv"))
obesidad = pd.read_csv(os.path.join("obesidad.csv"))
gripe = pd.read_csv(os.path.join("annual-mortality-rate-from-seasonal-influenza-ages-65.csv"))

In [2]:
#LIMPIEZA DATA 1: Si mi variable seleccionada es la población por cada país en el año 2020

# Como mi fuente me brinda información de diversas fechas, porcedo a filtrar los datos de 2020 directamente en el DataFrame original
poblacion = poblacion[poblacion['year'] == 2020]

# Cambio en la selección de columnas específicas, por el nombre de columna "population"
poblacion = poblacion[['country', 'population']].rename(columns={'population': 'Population_in_2020'})
poblacion.reset_index(drop=True, inplace=True)

# Se hace cambio del nombre de la columna "country" a "Country" en el DataFrame original
poblacion.rename(columns={'country': 'Country'}, inplace=True)

# Se realiza el cambio del texto en la columna "Country" de minúsculas a mayúsculas en el DataFrame original
poblacion['Country'] = poblacion['Country'].str.upper()



In [3]:
#Revisando si la primera data está bien, obtenemos el siguiente resultado
poblacion

,Country,Population_in_2020
0,AFGHANISTAN,3.897224e+07
1,AFRICA,1.360677e+09
2,AFRICA (GCP),NaN
3,ALAND ISLANDS,NaN
4,ALBANIA,2.866850e+06
...,...,...
266,WESTERN SAHARA,5.560600e+05
267,WORLD,7.840953e+09
268,YEMEN,3.228404e+07
269,ZAMBIA,1.892772e+07


In [4]:
#LIMPIAR DATA 2: índice de obesidad en adultos

# Reiniciar los índices después de ordenar, ya que vemos como estos mantienen las posiciones anteriores
obesidad.reset_index(drop=True, inplace=True)

# Se elimina la columna "Pos" y también es necesario cambiar el nombre de "Name" a "Country"
obesidad = obesidad.drop(columns=['Pos'])
obesidad = obesidad.rename(columns={'Name': 'Country'})

# Se cambia el nombre de la columna "Value" al nombre la variable para esta fuente "Obesity_adult_prevalence_rate"
obesidad = obesidad.rename(columns={'Value': 'Obesity_adult_prevalence_rate'})

# Se realiza el cambio del texto en la columna "Country", de minúsculas a mayúsculas
obesidad['Country'] = obesidad['Country'].str.upper()

#Finalmente se obtiene lo siguiente
obesidad

,Country,Obesity_adult_prevalence_rate
0,AMERICAN SAMOA,74.6
1,NAURU,71.1
2,COOK ISLANDS,63.7
3,TOKELAU,63.4
4,TONGA,57.6
...,...,...
186,MADAGASCAR,1.6
187,ERITREA,1.5
188,NEPAL,1.4
189,BANGLADESH,1.1


In [5]:
#LIMPIEZA DATA 3: El número anual de muertes por cada 100.000 personas en el grupo de edad de mayores de 65 años

# Cambiar el nombre de la columna "Entity" a "Country"
gripe = gripe.rename(columns={'Entity': 'Country'})

# Cambiar el texto en la columna "Country" de minúsculas a mayúsculas
gripe['Country'] = gripe['Country'].str.upper()


# Eliminar la columna "Year" y "Code"
gripe = gripe.drop(columns=['Year'])
gripe = gripe.drop(columns=['Code'])


# Cambiar el nombre de la columna "rate over 65" a "Country"
gripe = gripe.rename(columns={'rate over65': 'numero_anual_muertes'})



In [6]:
gripe

,Country,numero_anual_muertes
0,AFGHANISTAN,62.57
1,AFRICA,63.92
2,ALBANIA,29.59
3,ALGERIA,54.98
4,AMERICAS,53.60
...,...,...
195,WESTERN PACIFIC,43.45
196,WORLD,53.40
197,YEMEN,58.73
198,ZAMBIA,54.65


In [7]:
###########################################################################################################################

In [8]:
############################### PRIMER THEFUZZE #################################################

In [9]:
# Reviso los paises de cada una de mis datas, ver cuantos son distintos
OnlyPoblacion=set(poblacion.Country)-set(obesidad.Country)
OnlyObesidad=set(obesidad.Country)-set(poblacion.Country)


In [10]:
from thefuzz import process as fz
# Realizar los cambios
changeMerge1={f:fz.extractOne(f, OnlyPoblacion)[0] 
                 for f in sorted(OnlyObesidad)
                 if fz.extractOne(f, OnlyPoblacion)[1] >=90}
changeMerge1

{'BAHAMAS, THE': 'BAHAMAS',
 'CONGO, DEMOCRATIC REPUBLIC OF THE': 'DEMOCRATIC REPUBLIC OF CONGO',
 'CONGO, REPUBLIC OF THE': 'CONGO',
 'GAMBIA, THE': 'GAMBIA',
 'KOREA, NORTH': 'NORTH KOREA',
 'KOREA, SOUTH': 'SOUTH KOREA',
 'MACEDONIA': 'NORTH MACEDONIA'}

In [11]:
obesidad.Country.replace(to_replace=changeMerge1,inplace=True)

In [12]:
#######################SEGUNDO THEFUZZE ##################

In [13]:
#segundo filtro
OnlyPoblacion=set(poblacion.Country)-set(obesidad.Country)
OnlyObesidad=set(obesidad.Country)-set(poblacion.Country)
[(f,fz.extractOne(f, OnlyPoblacion)) for f in sorted(OnlyObesidad)]

[('BURMA', ('BERMUDA', 67)),
 ('CABO VERDE', ('CAPE VERDE', 80)),
 ('CZECH REPUBLIC', ('CZECHIA', 64)),
 ('MICRONESIA, FEDERATED STATES OF', ('MICRONESIA (COUNTRY)', 86)),
 ('SWAZILAND', ('TURKS AND CAICOS ISLANDS', 60)),
 ('TIMOR-LESTE', ('EAST TIMOR', 82)),
 ('TOKELAU', ('EUROPEAN UNION (27) (GCP)', 51))]

In [14]:
changeMerge2={f:fz.extractOne(f, OnlyPoblacion)[0] 
                 for f in sorted(OnlyObesidad)
                 if 74<=fz.extractOne(f, OnlyPoblacion)[1]}

In [15]:
changeMerge2

{'CABO VERDE': 'CAPE VERDE',
 'MICRONESIA, FEDERATED STATES OF': 'MICRONESIA (COUNTRY)',
 'TIMOR-LESTE': 'EAST TIMOR'}

In [16]:
obesidad.Country.replace(to_replace=changeMerge2,inplace=True)

In [17]:
########################## TERCER THEFUZZE ##################################################

In [18]:
OnlyPoblacion=set(poblacion.Country)-set(obesidad.Country)
OnlyObesidad=set(obesidad.Country)-set(poblacion.Country)
[(f,fz.extractOne(f, OnlyPoblacion)) for f in sorted(OnlyObesidad)]

[('BURMA', ('BERMUDA', 67)),
 ('CZECH REPUBLIC', ('CZECHIA', 64)),
 ('SWAZILAND', ('TURKS AND CAICOS ISLANDS', 60)),
 ('TOKELAU', ('EUROPEAN UNION (27) (GCP)', 51))]

In [19]:
changesMerge3={f:fz.extractOne(f, OnlyPoblacion)[0] 
                 for f in sorted(OnlyObesidad)
                 if 54==fz.extractOne(f, OnlyPoblacion)[1]}

In [20]:
changesMerge3

{}

In [21]:
obesidad.Country.replace(to_replace=changeMerge2,inplace=True)

In [22]:
#Realizaré el primer merge entre mi dos de mis datas
merge_data1=poblacion.merge(obesidad,left_on='Country',right_on='Country')

# Revisando el resultado del primer merge
merge_data1

,Country,Population_in_2020,Obesity_adult_prevalence_rate
0,AFGHANISTAN,38972236.0,2.2
1,ALBANIA,2866850.0,21.3
2,ALGERIA,43451668.0,16.0
3,AMERICAN SAMOA,46216.0,74.6
4,ANDORRA,77723.0,25.2
...,...,...,...
182,VENEZUELA,28490458.0,30.3
183,VIETNAM,96648680.0,1.7
184,YEMEN,32284044.0,14.5
185,ZAMBIA,18927716.0,3.6


In [23]:
########################## THEFUZZE 1 ###########################
OnlyMergeData1=set(merge_data1.Country)-set(gripe.Country)
OnlyGripe=set(merge_data1.Country)-set(gripe.Country)

changeMergeDataP={f:fz.extractOne(f, OnlyMergeData1)[0] 
                 for f in sorted(OnlyGripe)
                 if fz.extractOne(f, OnlyMergeData1)[1] >=90}


In [24]:
changeMergeDataP

{'AMERICAN SAMOA': 'AMERICAN SAMOA',
 'ANDORRA': 'ANDORRA',
 'COOK ISLANDS': 'COOK ISLANDS',
 'DOMINICA': 'DOMINICA',
 'MARSHALL ISLANDS': 'MARSHALL ISLANDS',
 'NAURU': 'NAURU',
 'PALAU': 'PALAU',
 'SAINT KITTS AND NEVIS': 'SAINT KITTS AND NEVIS'}

In [25]:
gripe.replace(to_replace=changeMergeDataP,inplace=True)

In [26]:
################## SEGUNDO THEFUZZE ########################

In [27]:
OnlyMergeData1=set(merge_data1.Country)-set(gripe.Country)
OnlyGripe=set(merge_data1.Country)-set(gripe.Country)
[(f,fz.extractOne(f, OnlyMergeData1)) for f in sorted(OnlyGripe)]

[('AMERICAN SAMOA', ('AMERICAN SAMOA', 100)),
 ('ANDORRA', ('ANDORRA', 100)),
 ('COOK ISLANDS', ('COOK ISLANDS', 100)),
 ('DOMINICA', ('DOMINICA', 100)),
 ('MARSHALL ISLANDS', ('MARSHALL ISLANDS', 100)),
 ('NAURU', ('NAURU', 100)),
 ('PALAU', ('PALAU', 100)),
 ('SAINT KITTS AND NEVIS', ('SAINT KITTS AND NEVIS', 100))]

In [28]:
changeMergeDataP2={f:fz.extractOne(f, OnlyMergeData1)[0] 
                 for f in sorted(OnlyGripe)
                 if 74<=fz.extractOne(f, OnlyMergeData1)[1]}

In [29]:
changeMergeDataP2

{'AMERICAN SAMOA': 'AMERICAN SAMOA',
 'ANDORRA': 'ANDORRA',
 'COOK ISLANDS': 'COOK ISLANDS',
 'DOMINICA': 'DOMINICA',
 'MARSHALL ISLANDS': 'MARSHALL ISLANDS',
 'NAURU': 'NAURU',
 'PALAU': 'PALAU',
 'SAINT KITTS AND NEVIS': 'SAINT KITTS AND NEVIS'}

In [30]:
gripe.replace(to_replace=changeMergeDataP2,inplace=True)

In [31]:
################TERCER THE FUZZE #######################

In [32]:
OnlyMergeData1=set(merge_data1.Country)-set(gripe.Country)
OnlyGripe=set(merge_data1.Country)-set(gripe.Country)
[(f,fz.extractOne(f, OnlyMergeData1)) for f in sorted(OnlyGripe)]


[('AMERICAN SAMOA', ('AMERICAN SAMOA', 100)),
 ('ANDORRA', ('ANDORRA', 100)),
 ('COOK ISLANDS', ('COOK ISLANDS', 100)),
 ('DOMINICA', ('DOMINICA', 100)),
 ('MARSHALL ISLANDS', ('MARSHALL ISLANDS', 100)),
 ('NAURU', ('NAURU', 100)),
 ('PALAU', ('PALAU', 100)),
 ('SAINT KITTS AND NEVIS', ('SAINT KITTS AND NEVIS', 100))]

In [33]:
changeMergeDataP3={f:fz.extractOne(f, OnlyMergeData1)[0] 
                 for f in sorted(OnlyGripe)
                 if 54==fz.extractOne(f, OnlyMergeData1)[1]}

In [34]:
changeMergeDataP3

{}

In [35]:
gripe.replace(to_replace=changeMergeDataP3,inplace=True)

In [36]:
# Como las datas que debo hacer son cuatro, debo realizar un nuevo merge, esta vez entre otra de mis datas
# y el realizado anteriormente
merge_data2=merge_data1.merge(gripe,left_on='Country',right_on='Country')

#revisando el resultado de este segundo merge
merge_data2

,Country,Population_in_2020,Obesity_adult_prevalence_rate,numero_anual_muertes
0,AFGHANISTAN,38972236.0,2.2,62.57
1,ALBANIA,2866850.0,21.3,29.59
2,ALGERIA,43451668.0,16.0,54.98
3,ANGOLA,33428490.0,6.4,73.83
4,ANTIGUA AND BARBUDA,92672.0,25.6,42.39
...,...,...,...,...
174,VENEZUELA,28490458.0,30.3,55.91
175,VIETNAM,96648680.0,1.7,37.39
176,YEMEN,32284044.0,14.5,58.73
177,ZAMBIA,18927716.0,3.6,54.65


In [37]:
######## PRIMER THEFUZZE ###########################

In [38]:
# Viendo las diferencias
OnlyMergeData2=set(merge_data2.Country)-set(country_isos.Country)
OnlyIsos=set(country_isos.Country)-set(merge_data2.Country)

changeMergeDataFinal={f:fz.extractOne(f, OnlyMergeData2)[0] 
                 for f in sorted(OnlyIsos)
                 if fz.extractOne(f, OnlyMergeData2)[1] >=90}

In [39]:
changeMergeDataFinal

{'BAHAMAS (THE)': 'BAHAMAS',
 'BOLIVIA (PLURINATIONAL STATE OF)': 'BOLIVIA',
 'BRUNEI DARUSSALAM': 'BRUNEI',
 'CENTRAL AFRICAN REPUBLIC (THE)': 'CENTRAL AFRICAN REPUBLIC',
 'COMOROS (THE)': 'COMOROS',
 'CONGO (THE DEMOCRATIC REPUBLIC OF THE)': 'DEMOCRATIC REPUBLIC OF CONGO',
 'CONGO (THE)': 'CONGO',
 "C√îTE D'IVOIRE": "COTE D'IVOIRE",
 'DOMINICAN REPUBLIC (THE)': 'DOMINICAN REPUBLIC',
 'GAMBIA (THE)': 'GAMBIA',
 'IRAN (ISLAMIC REPUBLIC OF)': 'IRAN',
 'MOLDOVA (THE REPUBLIC OF)': 'MOLDOVA',
 'NETHERLANDS (THE)': 'NETHERLANDS',
 'NIGER (THE)': 'NIGER',
 'PHILIPPINES (THE)': 'PHILIPPINES',
 'RUSSIAN FEDERATION (THE)': 'RUSSIA',
 'SOUTH SUDAN': 'SUDAN',
 'SUDAN (THE)': 'SUDAN',
 'TANZANIA, THE UNITED REPUBLIC OF': 'TANZANIA',
 'UNITED ARAB EMIRATES (THE)': 'UNITED ARAB EMIRATES',
 'UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND (THE)': 'UNITED KINGDOM',
 'UNITED STATES MINOR OUTLYING ISLANDS (THE)': 'UNITED STATES',
 'VIET NAM': 'VIETNAM'}

In [40]:
country_isos.replace(to_replace=changeMergeDataFinal,inplace=True)

In [41]:
####### SEGUNDO THEFUZZE #####

In [42]:
OnlyMergeData2=set(merge_data2.Country)-set(country_isos.Country)
OnlyIsos=set(country_isos.Country)-set(merge_data2.Country)

In [43]:
changeMergeDataFinal2={f:fz.extractOne(f, OnlyMergeData2)[0] 
                 for f in sorted(OnlyIsos)
                 if 74<=fz.extractOne(f, OnlyMergeData2)[1]}

In [44]:
changeMergeDataFinal2

{'CABO VERDE': 'CAPE VERDE',
 'KOREA (THE REPUBLIC OF)': 'NORTH KOREA',
 'MICRONESIA (FEDERATED STATES OF)': 'MICRONESIA (COUNTRY)',
 'TIMOR-LESTE': 'EAST TIMOR'}

In [45]:
country_isos.replace(to_replace=changeMergeDataFinal2,inplace=True)

In [46]:
# Finalmente, tenemos el merge del resultado de la unión de mis propias datas con las brindadas por el profesor 
merge_final=country_isos.merge(merge_data2,left_on='Country',right_on='Country')

# Revisando nuestro resultado, tenemos
merge_final

,Country,Officialstatename,InternetccTLD,iso2,iso3,Population_in_2020,Obesity_adult_prevalence_rate,numero_anual_muertes
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,38972236.0,2.2,62.57
1,ALBANIA,The Republic of Albania,.al,AL,ALB,2866850.0,21.3,29.59
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,43451668.0,16.0,54.98
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,33428490.0,6.4,73.83
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,92672.0,25.6,42.39
...,...,...,...,...,...,...,...,...
162,UZBEKISTAN,The Republic of Uzbekistan,.uz,UZ,UZB,33526662.0,15.1,37.22
163,VIETNAM,The Socialist Republic of Viet Nam,.vn,VN,VNM,96648680.0,1.7,37.39
164,YEMEN,The Republic of Yemen,.ye,YE,YEM,32284044.0,14.5,58.73
165,ZAMBIA,The Republic of Zambia,.zm,ZM,ZMB,18927716.0,3.6,54.65


In [47]:
#creamos una copia para realizar una vista de lo que serían la data si eliminamos los países que contienen NaN en alguna
# de sus celdas
merge_final_copia = merge_final.copy()
filas_con_nan = merge_final_copia.isna().any(axis=1)
filas_con_nan = merge_final_copia[filas_con_nan]

filas_con_nan
merge_final_copia.dropna(axis=0, how='any', inplace=True)

In [48]:
# Ahora, con toda mi data, puedo realizar la limpieza final

# Quiero asegurarme que los datos en mi columnas sean de un tipo de valor coherente (Números)
# por lo que es necesario deshacerme de valores como NaN, que pueden ser un obstáculo en la aplicación de mi nueva data
merge_final.replace(np.nan,0,inplace=True) 

# Revisando el merge final
merge_final

,Country,Officialstatename,InternetccTLD,iso2,iso3,Population_in_2020,Obesity_adult_prevalence_rate,numero_anual_muertes
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,38972236.0,2.2,62.57
1,ALBANIA,The Republic of Albania,.al,AL,ALB,2866850.0,21.3,29.59
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,43451668.0,16.0,54.98
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,33428490.0,6.4,73.83
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,92672.0,25.6,42.39
...,...,...,...,...,...,...,...,...
162,UZBEKISTAN,The Republic of Uzbekistan,.uz,UZ,UZB,33526662.0,15.1,37.22
163,VIETNAM,The Socialist Republic of Viet Nam,.vn,VN,VNM,96648680.0,1.7,37.39
164,YEMEN,The Republic of Yemen,.ye,YE,YEM,32284044.0,14.5,58.73
165,ZAMBIA,The Republic of Zambia,.zm,ZM,ZMB,18927716.0,3.6,54.65


In [49]:
# Cambiando el tipo de dato de las celdas de las columnas de mi data
merge_final['Obesity_adult_prevalence_rate'] = merge_final['Obesity_adult_prevalence_rate'].astype(str)
merge_final['Population_in_2020'] = merge_final['Population_in_2020'].astype(str)
merge_final['numero_anual_muertes'] = merge_final['numero_anual_muertes'].astype(str)

# Nos aseguramos de eliminar los espacios en blanco
merge_final.Obesity_adult_prevalence_rate.str.contains(pat=r'\d',regex=True)
merge_final.Population_in_2020.str.contains(pat=r'\d',regex=True)
merge_final.numero_anual_muertes.str.contains(pat=r'\d',regex=True)

# Verificamos la aplicación de los códigos
merge_final

,Country,Officialstatename,InternetccTLD,iso2,iso3,Population_in_2020,Obesity_adult_prevalence_rate,numero_anual_muertes
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af,AF,AFG,38972236.0,2.2,62.57
1,ALBANIA,The Republic of Albania,.al,AL,ALB,2866850.0,21.3,29.59
2,ALGERIA,The People's Democratic Republic of Algeria,.dz,DZ,DZA,43451668.0,16.0,54.98
3,ANGOLA,The Republic of Angola,.ao,AO,AGO,33428490.0,6.4,73.83
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag,AG,ATG,92672.0,25.6,42.39
...,...,...,...,...,...,...,...,...
162,UZBEKISTAN,The Republic of Uzbekistan,.uz,UZ,UZB,33526662.0,15.1,37.22
163,VIETNAM,The Socialist Republic of Viet Nam,.vn,VN,VNM,96648680.0,1.7,37.39
164,YEMEN,The Republic of Yemen,.ye,YE,YEM,32284044.0,14.5,58.73
165,ZAMBIA,The Republic of Zambia,.zm,ZM,ZMB,18927716.0,3.6,54.65


In [50]:
# Reemplazamos los valores en blanco, por guiones bajos
merge_final.rename(columns={'Officialstatename': 'Official_State_Name'}, inplace=True)
merge_final['Official_State_Name']=merge_final['Official_State_Name'].str.replace(' ','_',regex=True)
merge_final['Country']=merge_final['Country'].str.replace(' ','_',regex=True)

#Revisamos nuestra data
merge_final

,Country,Official_State_Name,InternetccTLD,iso2,iso3,Population_in_2020,Obesity_adult_prevalence_rate,numero_anual_muertes
0,AFGHANISTAN,The_Islamic_Republic_of_Afghanistan,.af,AF,AFG,38972236.0,2.2,62.57
1,ALBANIA,The_Republic_of_Albania,.al,AL,ALB,2866850.0,21.3,29.59
2,ALGERIA,The_People's_Democratic_Republic_of_Algeria,.dz,DZ,DZA,43451668.0,16.0,54.98
3,ANGOLA,The_Republic_of_Angola,.ao,AO,AGO,33428490.0,6.4,73.83
4,ANTIGUA_AND_BARBUDA,Antigua_and_Barbuda,.ag,AG,ATG,92672.0,25.6,42.39
...,...,...,...,...,...,...,...,...
162,UZBEKISTAN,The_Republic_of_Uzbekistan,.uz,UZ,UZB,33526662.0,15.1,37.22
163,VIETNAM,The_Socialist_Republic_of_Viet_Nam,.vn,VN,VNM,96648680.0,1.7,37.39
164,YEMEN,The_Republic_of_Yemen,.ye,YE,YEM,32284044.0,14.5,58.73
165,ZAMBIA,The_Republic_of_Zambia,.zm,ZM,ZMB,18927716.0,3.6,54.65


In [51]:
# Eliminar espacios en blanco
merge_final=merge_final.rename(columns=lambda x:x.strip())

# Finalmente, nuestro resultado final es el siguiente, considerando que los valores de NaN pasan a ser 0
merge_final

,Country,Official_State_Name,InternetccTLD,iso2,iso3,Population_in_2020,Obesity_adult_prevalence_rate,numero_anual_muertes
0,AFGHANISTAN,The_Islamic_Republic_of_Afghanistan,.af,AF,AFG,38972236.0,2.2,62.57
1,ALBANIA,The_Republic_of_Albania,.al,AL,ALB,2866850.0,21.3,29.59
2,ALGERIA,The_People's_Democratic_Republic_of_Algeria,.dz,DZ,DZA,43451668.0,16.0,54.98
3,ANGOLA,The_Republic_of_Angola,.ao,AO,AGO,33428490.0,6.4,73.83
4,ANTIGUA_AND_BARBUDA,Antigua_and_Barbuda,.ag,AG,ATG,92672.0,25.6,42.39
...,...,...,...,...,...,...,...,...
162,UZBEKISTAN,The_Republic_of_Uzbekistan,.uz,UZ,UZB,33526662.0,15.1,37.22
163,VIETNAM,The_Socialist_Republic_of_Viet_Nam,.vn,VN,VNM,96648680.0,1.7,37.39
164,YEMEN,The_Republic_of_Yemen,.ye,YE,YEM,32284044.0,14.5,58.73
165,ZAMBIA,The_Republic_of_Zambia,.zm,ZM,ZMB,18927716.0,3.6,54.65


In [52]:
merge_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Country                        167 non-null    object
 1   Official_State_Name            167 non-null    object
 2   InternetccTLD                  167 non-null    object
 3   iso2                           167 non-null    object
 4   iso3                           167 non-null    object
 5   Population_in_2020             167 non-null    object
 6   Obesity_adult_prevalence_rate  167 non-null    object
 7   numero_anual_muertes           167 non-null    object
dtypes: object(8)
memory usage: 10.6+ KB


In [53]:
# Arreglando el Dtype de algunas columnas
merge_final['Population_in_2020'] = merge_final['Population_in_2020'].astype('float64')
merge_final['Obesity_adult_prevalence_rate'] = merge_final['Obesity_adult_prevalence_rate'].astype('float64')
merge_final['numero_anual_muertes'] = merge_final['numero_anual_muertes'].astype('float64')

In [54]:
merge_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Country                        167 non-null    object 
 1   Official_State_Name            167 non-null    object 
 2   InternetccTLD                  167 non-null    object 
 3   iso2                           167 non-null    object 
 4   iso3                           167 non-null    object 
 5   Population_in_2020             167 non-null    float64
 6   Obesity_adult_prevalence_rate  167 non-null    float64
 7   numero_anual_muertes           167 non-null    float64
dtypes: float64(3), object(5)
memory usage: 10.6+ KB


In [55]:
merge_final_copia.rename(columns={'Officialstatename': 'Official_State_Name'}, inplace=True)
merge_final_copia['Official_State_Name']=merge_final_copia['Official_State_Name'].str.replace(' ','_',regex=True)
merge_final_copia['Country']=merge_final_copia['Country'].str.replace(' ','_',regex=True)
merge_final_copia['Population_in_2020'] = merge_final_copia['Population_in_2020'].astype('float64')
merge_final_copia['Obesity_adult_prevalence_rate'] = merge_final_copia['Obesity_adult_prevalence_rate'].astype('float64')
merge_final_copia['numero_anual_muertes'] = merge_final_copia['numero_anual_muertes'].astype('float64')
merge_final_copia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 165 entries, 0 to 166
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Country                        165 non-null    object 
 1   Official_State_Name            165 non-null    object 
 2   InternetccTLD                  165 non-null    object 
 3   iso2                           165 non-null    object 
 4   iso3                           165 non-null    object 
 5   Population_in_2020             165 non-null    float64
 6   Obesity_adult_prevalence_rate  165 non-null    float64
 7   numero_anual_muertes           165 non-null    float64
dtypes: float64(3), object(5)
memory usage: 11.6+ KB


In [56]:
merge_final_copia

,Country,Official_State_Name,InternetccTLD,iso2,iso3,Population_in_2020,Obesity_adult_prevalence_rate,numero_anual_muertes
0,AFGHANISTAN,The_Islamic_Republic_of_Afghanistan,.af,AF,AFG,38972236.0,2.2,62.57
1,ALBANIA,The_Republic_of_Albania,.al,AL,ALB,2866850.0,21.3,29.59
2,ALGERIA,The_People's_Democratic_Republic_of_Algeria,.dz,DZ,DZA,43451668.0,16.0,54.98
3,ANGOLA,The_Republic_of_Angola,.ao,AO,AGO,33428490.0,6.4,73.83
4,ANTIGUA_AND_BARBUDA,Antigua_and_Barbuda,.ag,AG,ATG,92672.0,25.6,42.39
...,...,...,...,...,...,...,...,...
162,UZBEKISTAN,The_Republic_of_Uzbekistan,.uz,UZ,UZB,33526662.0,15.1,37.22
163,VIETNAM,The_Socialist_Republic_of_Viet_Nam,.vn,VN,VNM,96648680.0,1.7,37.39
164,YEMEN,The_Republic_of_Yemen,.ye,YE,YEM,32284044.0,14.5,58.73
165,ZAMBIA,The_Republic_of_Zambia,.zm,ZM,ZMB,18927716.0,3.6,54.65


In [57]:
# Se tienen dos resultados merge_final_copia y merge_final:

# Considerando eliminar los países que contenían NaN
merge_final_copia

,Country,Official_State_Name,InternetccTLD,iso2,iso3,Population_in_2020,Obesity_adult_prevalence_rate,numero_anual_muertes
0,AFGHANISTAN,The_Islamic_Republic_of_Afghanistan,.af,AF,AFG,38972236.0,2.2,62.57
1,ALBANIA,The_Republic_of_Albania,.al,AL,ALB,2866850.0,21.3,29.59
2,ALGERIA,The_People's_Democratic_Republic_of_Algeria,.dz,DZ,DZA,43451668.0,16.0,54.98
3,ANGOLA,The_Republic_of_Angola,.ao,AO,AGO,33428490.0,6.4,73.83
4,ANTIGUA_AND_BARBUDA,Antigua_and_Barbuda,.ag,AG,ATG,92672.0,25.6,42.39
...,...,...,...,...,...,...,...,...
162,UZBEKISTAN,The_Republic_of_Uzbekistan,.uz,UZ,UZB,33526662.0,15.1,37.22
163,VIETNAM,The_Socialist_Republic_of_Viet_Nam,.vn,VN,VNM,96648680.0,1.7,37.39
164,YEMEN,The_Republic_of_Yemen,.ye,YE,YEM,32284044.0,14.5,58.73
165,ZAMBIA,The_Republic_of_Zambia,.zm,ZM,ZMB,18927716.0,3.6,54.65


In [58]:
# considerando el valor de NaN como 0
merge_final

,Country,Official_State_Name,InternetccTLD,iso2,iso3,Population_in_2020,Obesity_adult_prevalence_rate,numero_anual_muertes
0,AFGHANISTAN,The_Islamic_Republic_of_Afghanistan,.af,AF,AFG,38972236.0,2.2,62.57
1,ALBANIA,The_Republic_of_Albania,.al,AL,ALB,2866850.0,21.3,29.59
2,ALGERIA,The_People's_Democratic_Republic_of_Algeria,.dz,DZ,DZA,43451668.0,16.0,54.98
3,ANGOLA,The_Republic_of_Angola,.ao,AO,AGO,33428490.0,6.4,73.83
4,ANTIGUA_AND_BARBUDA,Antigua_and_Barbuda,.ag,AG,ATG,92672.0,25.6,42.39
...,...,...,...,...,...,...,...,...
162,UZBEKISTAN,The_Republic_of_Uzbekistan,.uz,UZ,UZB,33526662.0,15.1,37.22
163,VIETNAM,The_Socialist_Republic_of_Viet_Nam,.vn,VN,VNM,96648680.0,1.7,37.39
164,YEMEN,The_Republic_of_Yemen,.ye,YE,YEM,32284044.0,14.5,58.73
165,ZAMBIA,The_Republic_of_Zambia,.zm,ZM,ZMB,18927716.0,3.6,54.65


In [59]:
merge_final_copia.reset_index(drop=True, inplace=True)


In [60]:
merge_final_copia

,Country,Official_State_Name,InternetccTLD,iso2,iso3,Population_in_2020,Obesity_adult_prevalence_rate,numero_anual_muertes
0,AFGHANISTAN,The_Islamic_Republic_of_Afghanistan,.af,AF,AFG,38972236.0,2.2,62.57
1,ALBANIA,The_Republic_of_Albania,.al,AL,ALB,2866850.0,21.3,29.59
2,ALGERIA,The_People's_Democratic_Republic_of_Algeria,.dz,DZ,DZA,43451668.0,16.0,54.98
3,ANGOLA,The_Republic_of_Angola,.ao,AO,AGO,33428490.0,6.4,73.83
4,ANTIGUA_AND_BARBUDA,Antigua_and_Barbuda,.ag,AG,ATG,92672.0,25.6,42.39
...,...,...,...,...,...,...,...,...
160,UZBEKISTAN,The_Republic_of_Uzbekistan,.uz,UZ,UZB,33526662.0,15.1,37.22
161,VIETNAM,The_Socialist_Republic_of_Viet_Nam,.vn,VN,VNM,96648680.0,1.7,37.39
162,YEMEN,The_Republic_of_Yemen,.ye,YE,YEM,32284044.0,14.5,58.73
163,ZAMBIA,The_Republic_of_Zambia,.zm,ZM,ZMB,18927716.0,3.6,54.65
